<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/llm_makes_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
import os
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
# Set base parameters

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

LOAD_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/Scrapes"
LOAD_FILE = os.path.join(LOAD_PATH, "guideline_raw.json")

SAVE_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/JSON"
os.makedirs(SAVE_PATH, exist_ok=True)
SAVE_FILE = os.path.join(SAVE_PATH, "guideline_structured.json")

In [4]:
# Load LLM

"""
Focus should be on instruction-following models from Hugging Face
With free licence (Apache)
Qwen seems to have been trained on producing JSON formats
...allows for many tokens as input
...parameters are good balance between small and big
Should also check Llama offerings?
"""

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [5]:
# Test

## Should also carry out test prompt of transforming recommendations

text = "Should someone with a diagnosis of bipolar who is taking lithium be referred to secondary care if they are mildly irritable?"

inputs = tokenizer(text, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=500)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

Should someone with a diagnosis of bipolar who is taking lithium be referred to secondary care if they are mildly irritable? Or would it be better for them to see their GP for advice and support?
The decision on whether a person with bipolar disorder who is experiencing mild irritability while on lithium should be referred to secondary care or can be managed by their GP depends on several factors. Here are some considerations:

### Factors to Consider:
1. **Severity and Duration of Symptoms:**
   - Mild irritability might be within the range of expected mood fluctuations in bipolar disorder, especially during a depressive phase.
   - However, if the irritability is severe, persistent, or accompanied by other concerning symptoms (e.g., agitation, aggression), it may warrant further evaluation.

2. **Lithium Levels:**
   - Ensure that the lithium levels are within the therapeutic range. Fluctuations in lithium levels can affect mood stability.
   - If the levels are outside the optimal r

In [6]:
# Load JSON of raw recommendations

def load_json():
    try:
        with open(LOAD_FILE, "r", encoding="utf-8") as raw_recommendations:
            return json.load(raw_recommendations)
    except FileNotFoundError:
        raise FileNotFoundError(f'JSON file not found: {LOAD_FILE}')

raw_recommendations = load_json()

print(type(raw_recommendations))
print(len(raw_recommendations))
print(raw_recommendations[0])

<class 'list'>
136
{'heading_1': '1.1 Care for adults, children and young people across all phases of bipolar disorder', 'sub_heading_1': 'Treatment and support for specific populations', 'sub_heading_2': None, 'original_recommendation_number': '1.1.1', 'original_recommendation_text': 'Ensure that older people with bipolar disorder are offered the same range of treatments and services as younger people with bipolar disorder. '}


In [7]:
def construct_prompt(entity):

    """
    Given one recommendation entry {}, creates the prompt to extract one normalised JSON item
    """

    heading_1 = entity.get('heading_1')
    sub_heading_1 = entity.get('sub_heading_1')
    sub_heading_2 = entity.get('sub_heading_2')

    original_recommendation_number = entity.get('original_recommendation_number')
    original_recommendation_text = entity.get('original_recommendation_text')

    heading_context = " > ".join(h.strip() for h in [heading_1, sub_heading_1, sub_heading_2] if isinstance(h, str) and h.strip())

    return f"""
    You are extracting structured information from a NICE guideline recommendation.

    RULES:
    - output must be valid JSON only (no markdown)
    - do not invent clinical information, thresholds or populations; use only what is present
    - 'action' is a concrete imperative
    - if there is more than one action, retain all
    - 'conditionality' is determined by clauses that begin 'if...' or 'where...'
    - 'prohibitions' are 'do not', 'must not' and 'should not'
    - record urgency as 'True' if the text includes 'urgent', 'urgently', 'immediate' or 'immediately'
    - you MUST use 'null' if the information is not explicit in the recommendation or heading

    CONTEXT: {heading_context}

    RECOMMENDATION NUMBER: {original_recommendation_number}
    RECOMMENDATION TEXT: {original_recommendation_text}

    Produce JSON with exactly these keys:
    - action
    - scope
    - population
    - conditionality
    - prohibitions
    - urgency
    - original_recommendation_number
    - original_recommendation_text
    """


In [14]:
def run_llm_on_entity(tokenizer, model, entity):

    """
    Call the model on a single prompt using the prompt function
    Return model response
    """

    prompt = construct_prompt(entity)

    inputs = tokenizer(prompt,
                       return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             max_new_tokens=500,
                             do_sample=False) # deterministic decoding without random sampling
                                            # if removed, reinstate temperature / top_p / top_k

    llm_response = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[1]:],
                                          skip_special_tokens=True)

    return llm_response[0]

In [9]:
def convert_output_to_true_json(llm_response):
    """
    Takes output from run_llm_on_entity
    Turns it into a true JSON dictionary
    """

    llm_response = llm_response.strip()

    start = llm_response.find("{")
    end = llm_response.rfind("}")

    if start == -1 or end == -1 or end < start: # Where -1 is "not found" for str.find()
        raise ValueError("Could not find a JSON object in the LLM output.")

    json_string = llm_response[start:end + 1].strip()

    try:
        json_object = json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        raise

    if not isinstance(json_object, dict):
        raise TypeError(f"The object created by the function convert_output_to_true_json is not a JSON object. Instead it is a {type(json_object)}.")

    return json_object

In [13]:
def validate_json(json_object, required_keys):

    missing_keys = [k for k in required_keys if k not in json_object]
    if missing_keys:
        raise ValueError(f"Missing required keys: {missing_keys}")

    extra_keys = [k for k in json_object.keys() if k not in required_keys]
    if extra_keys:
        raise ValueError(f"Unexpected extra keys: {extra_keys}")

    for key, value in json_object.items():
        if value is None:
            continue
        if key == "urgency" and isinstance(value, bool):
            continue
        if isinstance(value, str):
            continue
        raise TypeError(f"Key '{key}' is of the wrong type, namely: {type(value)}.")

    return json_object

In [11]:
def orchestrate_create_json(raw_recommendations, tokenizer, model, save_file):

    compiled_recommendations = []

    errors = []

    required_keys = ["action",
                    "scope",
                    "population",
                    "conditionality",
                    "prohibitions",
                    "urgency",
                    "original_recommendation_number",
                    "original_recommendation_text"]

    for i, entity in enumerate(raw_recommendations):

        llm_output_text = run_llm_on_entity(tokenizer, model, entity)

        try:
            parsed_json = convert_output_to_true_json(llm_output_text)
            validate_json(parsed_json, required_keys)

        except Exception as e:
            print(f"Error {e} at point {i}")
            errors.append({"index": i, "error": str(e), "raw_llm_output": llm_output_text})
            continue

        compiled_recommendations.append(parsed_json)

    with open(save_file, "w", encoding="utf-8") as f:
        json.dump(compiled_recommendations, f, ensure_ascii=False, indent=2)

    print(f"Here is the list of json parsing errors: {errors}")

    return compiled_recommendations, errors

In [ ]:
orchestrate_create_json(raw_recommendations, tokenizer, model, SAVE_FILE)

JSON parsing error: Expecting value: line 7 column 20 (char 345)
Error Expecting value: line 7 column 20 (char 345) at point 8
JSON parsing error: Extra data: line 11 column 1 (char 661)
Error Extra data: line 11 column 1 (char 661) at point 9
Error Key 'action' is of the wrong type, namely: <class 'list'>. at point 12
Error Key 'action' is of the wrong type, namely: <class 'list'>. at point 13
JSON parsing error: Extra data: line 12 column 5 (char 518)
Error Extra data: line 12 column 5 (char 518) at point 18
